<a href="https://colab.research.google.com/github/HIsu1231/AI_INOVATION_SQUARE/blob/master/CNN_AdamOptimizer_3X3X32_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.4.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.4.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.4.0


In [2]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 42kB/s 
     |████████████████████████████████| 512kB 37.1MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 3.8MB 45.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=8ea53d5a62d8ad1644627734e18eefa3fed5fd8efd83538385e1f8783a786296
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninsta

In [12]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from datetime import datetime

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

print("")
print("train.num = ", mnist.train.num_examples,
      ", test.num = ", mnist.test.num_examples,
      ", validation.num = ", mnist.validation.num_examples)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz

train.num =  55000 , test.num =  10000 , validation.num =  5000


In [14]:
#hyper_parameter

learning_rate = 0.001
epochs = 30
batch_size = 100

i_nodes = mnist.train.images.shape[1]
o_nodes = mnist.train.labels.shape[1]

X = tf.placeholder(tf.float32, [None,i_nodes])
T = tf.placeholder(tf.float32, [None,o_nodes])

A1 = tf.reshape(X, [-1,28,28,1]) #image 28 X 28 X 1(color)

In [17]:
#컨볼루션층1, 3X3X32 필터
W2 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
b2 = tf.Variable(tf.random_normal([32]))

#28 X 28 X 32
C2 = tf.nn.conv2d(A1, W2, strides=[1,1,1,1], padding='SAME')

Z2 = tf.nn.relu(C2+b2)

#14 X 14 X 32
A2 = tf.nn.max_pool(Z2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

A2_flat = tf.reshape(A2, [-1, 14*14*32])

In [19]:
W3 = tf.Variable(tf.random_normal([14*14*32, 10], stddev=0.01))
b3 = tf.Variable(tf.random_normal([10]))

Z3 = tf.matmul(A2_flat,W3) + b3

y = tf.nn.softmax(Z3)

In [20]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z3, labels=T))

optimizer = tf.train.AdamOptimizer(learning_rate)

train = optimizer.minimize(loss)

In [22]:
predicted_val = tf.cast(tf.equal(tf.argmax(y,1), tf.argmax(T,1)), dtype=tf.float32)

accuracy = tf.reduce_mean(predicted_val)

In [23]:
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())

    start_time = datetime.now()

    for i in range(epochs):

        total_batch = int(mnist.train.num_examples / batch_size)

        for step in range(total_batch):

            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)

            loss_val, _ = sess.run([loss, train], feed_dict={X: batch_x_data, T: batch_t_data})

            if step % 100 == 0:
                print("epochs = ", i, ", step = ", step, ", loss value = ", loss_val)

    end_time = datetime.now()

    print("\nElapsed time = ", end_time - start_time)

    test_x_data = mnist.test.images
    test_t_data = mnist.test.labels

    accuracy_val = sess.run(accuracy, feed_dict={X: test_x_data, T: test_t_data})

    print("\nAccuracy = ", accuracy_val)


epochs =  0 , step =  0 , loss value =  2.8650248
epochs =  0 , step =  100 , loss value =  1.815392
epochs =  0 , step =  200 , loss value =  0.6366283
epochs =  0 , step =  300 , loss value =  0.61862963
epochs =  0 , step =  400 , loss value =  0.44295457
epochs =  0 , step =  500 , loss value =  0.3816613
epochs =  1 , step =  0 , loss value =  0.36100745
epochs =  1 , step =  100 , loss value =  0.36947384
epochs =  1 , step =  200 , loss value =  0.54065627
epochs =  1 , step =  300 , loss value =  0.40521434
epochs =  1 , step =  400 , loss value =  0.37382385
epochs =  1 , step =  500 , loss value =  0.49307024
epochs =  2 , step =  0 , loss value =  0.30342677
epochs =  2 , step =  100 , loss value =  0.45113605
epochs =  2 , step =  200 , loss value =  0.30152798
epochs =  2 , step =  300 , loss value =  0.4167991
epochs =  2 , step =  400 , loss value =  0.28647086
epochs =  2 , step =  500 , loss value =  0.4403656
epochs =  3 , step =  0 , loss value =  0.30358836
epochs =